In [20]:
# assumes you've downloaded data into ./data/source via ./download.sh

import os, glob, tables
import pandas as pd
from tqdm import tqdm as tqdm_base, tqdm_notebook as tqdm

tqdm_base.pandas()

In [2]:
position_files = glob.glob('./data/source/*_positions.csv')

In [3]:
dfs = [pd.read_csv(f) for f in tqdm(position_files)]
df = pd.concat(dfs)

In [4]:
df.shape

(5162629, 8)

In [5]:
df.head()

,route_id,timestamp,trip_id,vehicle_id,vehicle_lat,vehicle_long,time_utc,time_pct
0,100162,1515172533,34746138,7222,47.615437,-122.195480,2018-01-05 17:15:33,2018-01-05 09:15:33-08:00
1,100268,1515172609,34939262,6845,47.664951,-122.306503,2018-01-05 17:16:49,2018-01-05 09:16:49-08:00
2,100223,1515172627,34742858,2708,47.611298,-122.337379,2018-01-05 17:17:07,2018-01-05 09:17:07-08:00
3,100214,1515172587,34939012,6886,47.758350,-122.249443,2018-01-05 17:16:27,2018-01-05 09:16:27-08:00
4,100204,1515172591,34938946,3646,47.776638,-122.313705,2018-01-05 17:16:31,2018-01-05 09:16:31-08:00


In [6]:
df.drop_duplicates(inplace=True)

In [7]:
df.shape

(5154140, 8)

In [8]:
df.dtypes

route_id          int64
timestamp         int64
trip_id           int64
vehicle_id        int64
vehicle_lat     float64
vehicle_long    float64
time_utc         object
time_pct         object
dtype: object

In [9]:
df['timestamp_tzone'] = df['time_pct'].progress_apply(pd.to_datetime)

100%|██████████| 5154140/5154140 [05:36<00:00, 15317.33it/s]


In [10]:
df.dtypes

route_id                    int64
timestamp                   int64
trip_id                     int64
vehicle_id                  int64
vehicle_lat               float64
vehicle_long              float64
time_utc                   object
time_pct                   object
timestamp_tzone    datetime64[ns]
dtype: object

In [ ]:
# output cleared for notebook (just cleared error output), but run this anyway
df.set_index('timestamp_tzone', inplace=True)

In [16]:
df.sort_index(inplace=True)

In [17]:
df.head()

,route_id,timestamp,trip_id,vehicle_id,vehicle_lat,vehicle_long,time_utc,time_pct
timestamp_tzone,,,,,,,,
2017-12-31 23:55:06,100249,1514764506,34922827,7006,47.617992,-122.320847,2017-12-31 23:55:06,2017-12-31 15:55:06-08:00
2017-12-31 23:55:07,100146,1514764507,34763985,6905,47.598827,-122.328072,2017-12-31 23:55:07,2017-12-31 15:55:07-08:00
2017-12-31 23:55:08,100469,1514764508,34970012,7062,47.412113,-122.272278,2017-12-31 23:55:08,2017-12-31 15:55:08-08:00
2017-12-31 23:55:28,100259,1514764528,34771722,3676,47.706902,-122.317825,2017-12-31 23:55:28,2017-12-31 15:55:28-08:00
2017-12-31 23:55:28,100101,1514764528,34922364,2770,47.555485,-122.376259,2017-12-31 23:55:28,2017-12-31 15:55:28-08:00


In [21]:
df.to_hdf('./data/intermed/positions_201801.h5', key='df', mode='w')

In [23]:
df.loc['2018-01-01'].shape

(110963, 8)